In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import utils
import featurizer
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import classify
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import copy
CALIBRATION_FILE = "N_matrix_trial9.mat"
TEST_FOLDER = "test/"
TEST_FILE = TEST_FOLDER + "/sliding11.txt"

/Users/Can/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#First we need to featurize the experiment.
df = utils.process_data_files(TEST_FILE, CALIBRATION_FILE)
df_segs = featurizer.segment(df)
print len(df_segs)

test_data = []
for df_seg in df_segs:
    test_data.append(featurizer.featurize(df_seg))
    
# print len(test_data)
# print test_data[0].shape
test_data_copy = copy.deepcopy(test_data)

Index([u'time', u'Right Leg Pos', u'Left Leg Pos', u'Commanded Right Leg Pos',
       u'Commanded Left Leg Pos', u'DCR', u'DCL', u'GyroX', u'GyroY', u'GyroZ',
       u'AX', u'AY', u'AZ', u'RBEMF', u'LBEMF', u'VBatt', u'S1', u'S2', u'S3',
       u'S4', u'S5', u'S6', u'S7', u'S8'],
      dtype='object')
38


In [3]:
#Random Forest Prediction
random_forest_predictions = classify.predict_random_forests(test_data)


Running Random Forests...
CTL size: 117 ACT size: 99
10-fold CV Acc Mean:  0.930952380952 Scores:  [ 1.          1.          0.95454545  0.81818182  0.95454545  0.90909091
  0.86363636  0.95238095  0.85714286  1.        ]
OOB score: 0.935185185185
Feature Importances:
[('Fy_energy', 0.0013265310522812186), ('Mx_skew', 0.0013783577444485778), ('Mz_skew', 0.0015137866110879901), ('Fz_entropy', 0.0017981636746559832), ('AX_std', 0.00194313602163936), ('AX_mean', 0.0019915115710678103), ('Fz_skew', 0.0021401406669708141), ('M_mag_std', 0.0022446080689925317), ('Fy_kurtosis', 0.0024169805557389809), ('F_mag_kurtosis', 0.0026053581116786493), ('F_mag_max', 0.0029080844847794184), ('Mz_energy', 0.0029565375528493197), ('Fx_std', 0.0029848107165786038), ('My_skew', 0.003048513412340431), ('Mz_std', 0.003059946788579035), ('Fz_max', 0.0030874974999231791), ('AX_min', 0.0030964591584190055), ('Mz_min', 0.0032242057879693299), ('My_entropy', 0.0033420853826374593), ('M_mag_max', 0.00343668198672

In [ ]:
cdict = {
  'red'  :  [],
  'blue' :  []
}

for i in xrange(len(df_segs)):
    time_min = 
    color = 'b'
    if random_forest_predictions[i] == 1:
        color = 'r'
        
    plt.plot(df_segs[i]["time"], df_segs[i]["F_mag"],color=color)
    plt.xlabel("Time (s)")
    plt.ylabel("Force (N)")
    plt.show()

In [ ]:
utils.plot_columns(df, [["Fx", "Fy", "Fz"], "F_mag", ["Mx", "My", "Mz"], "M_mag", ["AX", "AY", "AZ"], "A_mag", ["GyroX", "GyroY", "GyroZ"], "Gyro_mag"], display=True, save_figure=False, output_filename="FMAG.png")


In [4]:
# XGB Predictions
test_data = copy.deepcopy(test_data_copy)
xgb_predictions = classify.predict_xgb_trees(test_data)


Running XGB Trees...
CTL size: 117 ACT size: 99
10-fold CV Acc Mean:  0.939350649351 Scores:  [ 0.90909091  1.          0.95454545  0.90909091  0.95454545  0.90909091
  1.          0.85714286  1.          0.9       ]
Feature Importances:
[('F_mag_mean', 0.0010448138107755729), ('My_entropy', 0.0014745461037863595), ('Mx_skew', 0.0017817423670558193), ('Fz_skew', 0.0018994986581687145), ('Fy_kurtosis', 0.0019843267679125055), ('Fy_energy', 0.0021311426289411205), ('AX_mean', 0.0021672557248117682), ('Fz_entropy', 0.0021890700940414205), ('F_mag_kurtosis', 0.0022474768727891708), ('Fx_energy', 0.0024601716689595979), ('Mz_energy', 0.0027438362062716408), ('Mz_max', 0.0031304453727235716), ('My_skew', 0.0036928140851114439), ('Mz_skew', 0.0038370145771720208), ('Mz_kurtosis', 0.0039726012051334685), ('AX_min', 0.0044577937831529572), ('Fy_std', 0.0045576699320952598), ('My_max', 0.0047549212581587284), ('M_mag_std', 0.0047628776447068118), ('Fz_max', 0.0048252134774070645), ('Fz_kurtosis

In [5]:
# SVC Predictions
test_data = copy.deepcopy(test_data_copy)
svc_predictions = classify.predict_svc(test_data)


Running SVC...
CTL size: 117 ACT size: 99
10-fold CV Acc Mean:  0.949112554113 Scores:  [ 0.90909091  0.95454545  0.90909091  1.          0.95454545  0.95454545
  0.95454545  0.95238095  0.95238095  0.95      ]
predicted labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [7]:
# dnn predictions
test_data = copy.deepcopy(test_data_copy)
test_data = np.array(test_data)
print test_data.shape
dnn_predictions = classify.predict_dnn(test_data)

(38, 70)

Running Neural Network...
CTL size: 117 ACT size: 99
Step #1, avg. loss: 1.01757
Step #101, epoch #50, avg. loss: 0.12096
Step #201, epoch #100, avg. loss: 0.01949
Step #301, epoch #150, avg. loss: 0.00737
Step #401, epoch #200, avg. loss: 0.00405
Step #501, epoch #250, avg. loss: 0.00264
Step #601, epoch #300, avg. loss: 0.00191
Step #701, epoch #350, avg. loss: 0.00147
Step #801, epoch #400, avg. loss: 0.00118
Step #901, epoch #450, avg. loss: 0.00097
Step #1, avg. loss: 1.12548
Step #101, epoch #50, avg. loss: 0.11390
Step #201, epoch #100, avg. loss: 0.01764
Step #301, epoch #150, avg. loss: 0.00679
Step #401, epoch #200, avg. loss: 0.00371
Step #501, epoch #250, avg. loss: 0.00243
Step #601, epoch #300, avg. loss: 0.00177
Step #701, epoch #350, avg. loss: 0.00134
Step #801, epoch #400, avg. loss: 0.00108
Step #901, epoch #450, avg. loss: 0.00089
Step #1, avg. loss: 1.07794
Step #101, epoch #50, avg. loss: 0.12374
Step #201, epoch #100, avg. loss: 0.02056
Step #301, epoch

In [ ]:
[1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0]